In [1]:
from pynq import Overlay,allocate
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [2]:
ol = Overlay("design_1.bit")

In [3]:
ol.ip_dict.keys()
# ol?

dict_keys(['radar_gen_0', 'gpio', 'dma_radar_gen', 'processing_system7_0'])

In [4]:
dma = ol.dma_radar_gen
dma_send = ol.dma_radar_gen.sendchannel
dma_recv = ol.dma_radar_gen.recvchannel

In [5]:
print(dma_recv.idle,dma_send.idle)

False False


In [6]:
# data_size = 10**6

# input_buffer = allocate(shape=(data_size,), dtype=np.uint32)
# output_buffer = allocate(shape=(data_size,), dtype=np.uint32)

In [7]:
# #0,1 - signal
# #2,3 - up/down
# #4,5 - inc
# #6,7 - cs
# pwd = 10000
# pri = 100000
# for i in range(data_size):
# #     print(round((i/data_size),4),i)
# #     clear_output(wait=True)
#     if((i%pri)>=0 and (i%pri)<pwd):
#         input_buffer_s[i]=1
#     else:
#         input_buffer_s[i]=0
# print(1)
# inc_counter=0
# upd_counter=0
# for i in range(data_size):
# #     print(round((i/data_size),4),i)
# #     clear_output(wait=True)
#     if((i%pri+pwd)>=0 and (i%pri+pwd)<pwd):
#         input_buffer[i]=1
#         inc_counter=0
#     else:
#         if(inc_counter==0):
#             if(upd_counter==0):
#                 upd_counter=1
#                 input_buffer[i]=2
#             else:
#                 upd_counter=0
#                 input_buffer[i]=3
#         else:
#             if(inc_counter<100):
#                 inc_counter+=1
#                 if(inc_counter%2==0):
#                     input_buffer[i]=4
#                 else:
#                     input_buffer[i]=5
#             else:
#                 input_buffer[i]=0
#     if(i==0):
#         input_buffer[i]=7
# print(2)

In [281]:
# data_size = 1*(10**3)
pwd=20
data_size = (pwd*100)
#sig_ud_inc_cs
input_buffer = allocate(shape=(data_size,), dtype=np.uint32)
output_buffer = allocate(shape=(data_size,), dtype=np.uint32)

print(1)

inc_counter=0
upd_counter=0
counter_pwd=10**6
for i in range(data_size):
#     print(round((i/data_size),4),i)
#     clear_output(wait=True)
#     if(i%counter_pwd<(counter_pwd*0.1)):
    if(i%pwd<pwd//2):
        input_buffer[i]=int('1010', 2)
    else:
        input_buffer[i]=int('1000', 2)
print(2)

1
2


In [282]:
dma_send.transfer(input_buffer)
dma_recv.transfer(output_buffer)

# output_buffer

In [267]:
for i in range(10000):
#     clear_output(wait=True)
#     print(i)
    dma_send.transfer(input_buffer)
    dma_recv.transfer(output_buffer)
#     break
    while(not dma_send.idle):
        pass

# dma_send.idle
# output_buffer

In [ ]:
# for i in range(1000):
#     clear_output(wait=True)
#     print(i)
#     dma_send.transfer(input_buffer)
#     dma_recv.transfer(output_buffer)
# #     break
#     while(not dma_send.idle):
#         pass

# dma_send.idle

353


In [21]:
# for i in range(1000):
# #     clear_output(wait=True)
# #     print(i)
#     dma_send.transfer(input_buffer_s)
#     dma_recv.transfer(output_buffer)
# #     break
#     while(not dma_send.idle):
#         pass

# dma_send.idle

True

In [ ]:
# data_size = 1*(10**3)
pwd=500
add_time=pwd*100
data_size = add_time*20
#sig_ud_inc_cs
input_buffer_t = allocate(shape=(data_size,), dtype=np.uint32)
output_buffer_t = allocate(shape=(data_size,), dtype=np.uint32)

out=['1','1','1','1']
print(1)
count_p=0
count_n=0
count_l=0
res=50
current_res=50
# data_set=[]
curr_sig=0
prev_sig=0
flag=True
print(data_size)
i=0
add_counter=0
while (i <data_size):
    if(i%(data_size/4)>=(4*(data_size/4)/5) and i%(data_size/4)<((data_size/4))):
        if((i%(data_size/2))<(data_size/4)):
            curr_sig=2
#             print(2,end = '.')
#             data_set.append(2)
        else:
            curr_sig=1
#             print(1,end = '.')
#             data_set.append(1)
        out[0]='1'
    else:
        curr_sig=0
#         print(0,end = '.')
#         data_set.append(0)
        out[0]='0'
    if(curr_sig!=0):
        if(curr_sig==1):
            res=50
        else:
            res=100
    if(res==current_res):
        out[3]='1'
        flag=True
    else:
        if(flag):
            add_counter=0
            flag=False
            if(prev_sig == 0):
                print(i,current_res,res)
                if(i>pwd*100):
                    i-=pwd*100
                continue
        out[3]='0'
        if(current_res<res):
            if(out[1]=='1'):
                add_counter=0
            add_counter+=1
            if(add_counter>=pwd):
                add_counter=0
                current_res+=1
                if(res==current_res):
                    print(i,current_res)
                else:
                    print(i,current_res,end = ' | ')
            out[1]='0'
        else:
            if(out[1]=='0'):
                add_counter=0
            add_counter+=1
            if(add_counter>=pwd):
                add_counter=0
                current_res-=1
                if(res==current_res):
                    print(i,current_res)
                else:
                    print(i,current_res,end = ' | ')
            out[1]='1'
    if(out[3]=='0'):
        if(i%pwd<pwd/2):
            out[2]='1'
        else:
            out[2]='0'
    out[3]='0'
    input_buffer_t[i]=int(str(out[0]+out[1]+out[2]+out[3]), 2)
    prev_sig=curr_sig
    i+=1
print()
print(2)
dma_send.transfer(input_buffer_t)
dma_recv.transfer(output_buffer_t)

In [ ]:
dma_send.transfer(input_buffer_t_p)
dma_recv.transfer(output_buffer_t_p)
while(not dma_send.idle):
    pass
dma_send.transfer(input_buffer_t_p)
dma_recv.transfer(output_buffer_t_p)
while(not dma_send.idle):
    pass
dma_send.transfer(input_buffer_t_n)
dma_recv.transfer(output_buffer_t_n)
while(not dma_send.idle):
    pass
for i in range(100):
#     clear_output(wait=True)
#     print(i)
    dma_send.transfer(input_buffer_t)
    dma_recv.transfer(output_buffer_t)
#     break
    while(not dma_send.idle):
        pass
#     timess.sleep(0.001)

# dma_send.idle
# output_buffer

In [ ]:
for i in range(1000):
#     clear_output(wait=True)
#     print(i)
    dma_send.transfer(input_buffer_t_n)
    dma_recv.transfer(output_buffer_t_n)
#     break
    while(not dma_send.idle):
        pass
#     timess.sleep(0.001)
    dma_send.transfer(input_buffer_t_p)
    dma_recv.transfer(output_buffer_t_p)
#     break
    while(not dma_send.idle):
        pass
#     timess.sleep(0.001)

# dma_send.idle
# output_buffer

In [ ]:
# data_size = 1*(10**3)
pwd=13
data_size = (pwd*50)
#sig_ud_inc_cs
input_buffer_t_p = allocate(shape=(data_size,), dtype=np.uint32)
output_buffer_t_p = allocate(shape=(data_size,), dtype=np.uint32)

out=['1','0','1','0']
print(1)
for i in range(data_size):
#     print(round((i/data_size),4),i)
#     clear_output(wait=True)
#     if(i%counter_pwd<(counter_pwd*0.1)):
    out[0]='1'
    out[1]='0'
    out[3]='0'
    if(i%pwd<pwd//2):
        out[2]='1'
    else:
        out[2]='0'
    input_buffer_t_p[i]=int(str(out[0]+out[1]+out[2]+out[3]), 2)
print(2)
dma_send.transfer(input_buffer_t_p)
dma_recv.transfer(output_buffer_t_p)

# output_buffer

In [ ]:
# data_size = 1*(10**3)
#sig_ud_inc_cs
input_buffer_t_n = allocate(shape=(data_size,), dtype=np.uint32)
output_buffer_t_n = allocate(shape=(data_size,), dtype=np.uint32)

out=['1','0','1','0']
print(1)
for i in range(data_size):
#     print(round((i/data_size),4),i)
#     clear_output(wait=True)
#     if(i%counter_pwd<(counter_pwd*0.1)):
    out[0]='1'
    out[1]='1'
    out[3]='0'
    if(i%pwd<pwd//2):
        out[2]='1'
    else:
        out[2]='0'
    input_buffer_t_n[i]=int(str(out[0]+out[1]+out[2]+out[3]), 2)
print(2)
dma_send.transfer(input_buffer_t_n)
dma_recv.transfer(output_buffer_t_n)

# output_buffer

In [10]:
dma_send.idle

True

In [9]:
print(output_buffer)

PynqBuffer([ 500000000, 1000000000,          0, ...,          0,
                     0,          0], dtype=uint32)

In [ ]:
del input_buffer, output_buffer